In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from jupyter import REPO_ROOT
import os
import pwd

In [2]:
def get_username():
    return pwd.getpwuid( os.getuid() )[ 0 ]

In [3]:
conn = psycopg2.connect(
    database = "ergast_f1_stats",
    user = get_username(),
    host = 'localhost',
    port = '5432'
)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [4]:
TABLES = ["seasons", "circuits", "constructors", "races", "drivers", "status", "constructor_results", "constructor_standings", "lap_times", "pit_stops", "results", "driver_standings"]
QUALIFYING_TABLE = "qualifying"

In [5]:
cursor = conn.cursor()

def drop_table(table):
    print(f"DROPPING: {table}")
    cursor.execute(f"DROP TABLE {table} CASCADE")

def insert_table(table, cols_force_null=None):
    print(f"INSERTING: {table}")
    sql_copy = f'''
        COPY {table}
        FROM '{REPO_ROOT}/ergast-database-csv/{table}.csv' 
        DELIMITER ','
        CSV HEADER
    '''
    if cols_force_null:
        sql_copy += f'FORCE NULL {cols_force_null}'
    sql_copy += ";"
    print(sql_copy)
    print(cursor.execute(sql_copy))
    cursor.execute(f"SELECT * FROM {table} LIMIT 1;")
    print(cursor.fetchone())

In [ ]:
# Delete everything!
drop_table(QUALIFYING_TABLE)
for table in TABLES:
    drop_table(table)

DROPPING: qualifying
DROPPING: seasons
DROPPING: circuits
DROPPING: constructors
DROPPING: races
DROPPING: drivers
DROPPING: status
DROPPING: constructor_results
DROPPING: constructor_standings
DROPPING: lap_times
DROPPING: pit_stops
DROPPING: results
DROPPING: driver_standings


In [6]:
# Create tables and relations
with conn.cursor() as curs:
    curs.execute(open(f"{REPO_ROOT}/database-ddl-commands.ddl", "r").read())

In [7]:
# Fill the tables with data
for table in TABLES:
    insert_table(table)
insert_table(QUALIFYING_TABLE, "q1,q2,q3")

INSERTING: seasons

        COPY seasons
        FROM '/Users/drewgervang/Documents/Code/f1-track-models/ergast-database-csv/seasons.csv' 
        DELIMITER ','
        CSV HEADER
    ;
None
(2009, 'https://en.wikipedia.org/wiki/2009_Formula_One_season')
INSERTING: circuits

        COPY circuits
        FROM '/Users/drewgervang/Documents/Code/f1-track-models/ergast-database-csv/circuits.csv' 
        DELIMITER ','
        CSV HEADER
    ;
None
(1, 'albert_park', 'Albert Park Grand Prix Circuit', 'Melbourne', 'Australia', -37.8497, 144.968, 10.0, 'http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit')
INSERTING: constructors

        COPY constructors
        FROM '/Users/drewgervang/Documents/Code/f1-track-models/ergast-database-csv/constructors.csv' 
        DELIMITER ','
        CSV HEADER
    ;
None
(1, 'mclaren', 'McLaren', 'British', 'http://en.wikipedia.org/wiki/McLaren')
INSERTING: races

        COPY races
        FROM '/Users/drewgervang/Documents/Code/f1-track-models/er